# Continous Kitsu Scripts
* This notebook will run indefinitely. You must manually kill this notebook once a sufficient number of lists have been found

In [ ]:
import os
import subprocess
import time

In [ ]:
os.chdir("..")
cwd = os.getcwd()

In [ ]:
if os.path.exists("../../../data/proxies"):
    with open("../../../data/proxies/proxies.txt", "r") as f:
        NUM_PROXIES = len(f.readlines())
else:
    NUM_PROXIES = 1

In [ ]:
NUM_KITSU_ANIME_PARTITIONS = max(1, NUM_PROXIES * 2 // 3)
NUM_KITSU_MANGA_PARTITIONS = max(1, NUM_PROXIES - NUM_KITSU_ANIME_PARTITIONS)

In [ ]:
if NUM_KITSU_ANIME_PARTITIONS + NUM_KITSU_MANGA_PARTITIONS > NUM_PROXIES:
    API_PERIOD_MULT = 2
else:
    API_PERIOD_MULT = 1

In [ ]:
procs = []
def spawn_process(cmdlist):
    print(cmdlist)
    p = subprocess.Popen(cmdlist)
    procs.append(p)
    time.sleep(1)

In [ ]:
def spawn_kitsu_process(partition, num_partitions, mediatype, proxyid):
    cmdlist = [
        "papermill",
        f"{cwd}/GetUserMediaListsKitsu.ipynb",
        "/dev/null",        
        "-p",
        "PARTITION",
        str(partition),
        "-p",
        "NUM_PARTITIONS",
        str(num_partitions),
        "-p",
        "MEDIATYPE",
        mediatype,
        "-p",
        "PROXY_NUMBER",
        str(proxyid),
        "-p",
        "API_PERIOD_MULT",
        str(API_PERIOD_MULT),
    ]
    spawn_process(cmdlist)

In [ ]:
for i in range(NUM_KITSU_ANIME_PARTITIONS):
    spawn_kitsu_process(i, NUM_KITSU_ANIME_PARTITIONS, "anime", i)

In [ ]:
for i in range(NUM_KITSU_MANGA_PARTITIONS):
    spawn_kitsu_process(
        i, NUM_KITSU_MANGA_PARTITIONS, "manga", i + NUM_KITSU_ANIME_PARTITIONS
    )